In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    repo_dir = Path("mlfs-book")
    if repo_dir.exists():
        print(f"Repository already exists at {repo_dir.absolute()}")
        %cd mlfs-book
    else:
        print("Cloning repository...")
        !git clone https://github.com/featurestorebook/mlfs-book.git
        %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('titanic',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Local environment
Added the following directory to the PYTHONPATH: C:\Users\krist\Documents\GitHub\mlfs-book


In [2]:
from PIL import Image
import datetime
import requests
import xgboost as xgb
import os
import hopsworks
import pandas as pd
from mlfs import config
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
mr = project.get_model_registry()
model_reg = mr.get_model("titanic", version=1)
feature_view = model_reg.get_feature_view()

# Loading the XGBoost  model  from the saved model in the registry
model = xgb.XGBClassifier()
# Downloads the model to a local temp directory (model_dir)
model_dir = model_reg.download()
# Load XGBoost model from local temp directory
model.load_model(model_dir + "/titanic_model.json")

In [ ]:
feature_view.init_batch_scoring(training_dataset_version=1)

batch_data = feature_view.get_batch_data()
batch_data

In [ ]:
y_pred = model.predict(batch_data)
print("Did the last passenger survive?")
passenger_survived = y_pred[y_pred.size-1]
print("Yes, the last generated passenger survived!" if passenger_survived else "Unfortunately, the last generated passenger did not survive.")


In [ ]:
passenger_details = batch_data.iloc[-1]
def remove_last_line_from_string(s):
    return s[:s.rfind('\n')]
passenger_details = remove_last_line_from_string(str(passenger_details))

with open(f"{root_dir}/docs/_includes/titanic.txt", "w", newline="\n") as file:
    file.write(passenger_details)

# with open(f"{root_dir}/docs/_includes/titanic.html", "w", newline="\n") as file:
#     file.write(passenger_details)

In [ ]:
img_url = "https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/docs/titanic/assets/img/titanic_" + str(passenger_survived) + ".jpg"
img = Image.open(requests.get(img_url, stream=True).raw)            
img.save(f"{root_dir}/docs/titanic/assets/img/latest_titanic_survival.jpg")

In [ ]:
import json
survived_value = int(passenger_survived)
with open('prediction.json', 'w') as f:
    json.dump({'survived': survived_value, 'img_url': img_url}, f)

In [ ]:
from IPython.display import display
display(img)